# Probe Data - Map Matching
## Nick Paras | Kapil Garg

### Assignment 2

Input: Probe data and map [probe_data_map_matching.rar](https://canvas.northwestern.edu/courses/51440/files/3334329/download?wrap=1)

-The raw probe points in Germany collected in 9 months

-The link data for the links that probe points can be map-matched to.

Tasks:
-- map match probe points to road links

-- derive road slope for each road link

-- evaluate the derived road slope with the surveyed road slope in the link data file

**Please submit your code and slides presentation of your approach and results including evaluation comparing with the slopes in the link data file**

### Setup

We use **Python 3.6** and rely on the dependencies:
* numpy
* scikit-learn
* matplotlib
* pandas

We also use Jupyter Notebooks for our code and reports. For quick setup, please create a conda environment with the following:

    $ conda create --name probe-data pandas matplotlib numpy scikit-learn

and then activate the conda environment with

    $ source activate probe-data


In [3]:
# Imports
import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

# Constants
DATA_DIR = "probe_data_map_matching"

## Loading Probe Data for Map Matching

Here we'll load our data from the two csv's into Pandas DataFrames.

In [8]:
probe_headers = ["sampleID", 
                 "dateTime", 
                 "sourceCode", 
                 "latitude", 
                 "longitude", 
                 "altitude", 
                 "speed", 
                 "heading"]

probe_data = pd.read_csv(os.path.join(DATA_DIR, "Partition6467ProbePoints.csv"), header=None, names=probe_headers)

In [9]:
probe_data.head()

,sampleID,dateTime,sourceCode,latitude,longitude,altitude,speed,heading
0,3496,6/12/2009 6:12:49 AM,13,51.496868,9.386022,200,23,339
1,3496,6/12/2009 6:12:54 AM,13,51.496682,9.386157,200,10,129
2,3496,6/12/2009 6:12:59 AM,13,51.496705,9.386422,201,21,60
3,3496,6/12/2009 6:13:04 AM,13,51.496749,9.386840,201,0,360
4,3496,6/12/2009 6:13:09 AM,13,51.496864,9.387294,199,0,360


In [12]:
link_headers = ["linkPVID", 
                "refNodeID", 
                "nrefNodeID", 
                "length", 
                "functionalClass", 
                "directionOfTravel", 
                "speedCategory", 
                "fromRefSpeedLimit", 
                "toRefSpeedLimit", 
                "fromRefNumLanes", 
                "toRefNumLanes", 
                "multiDigitized", 
                "urban", 
                "timeZone", 
                "shapeInfo", 
                "curvatureInfo", 
                "slopeInfo"]

link_data = pd.read_csv(os.path.join(DATA_DIR, "Partition6467LinkData.csv"), header=None, names=link_headers, dtype=str)

In [13]:
link_data.head()

,linkPVID,refNodeID,nrefNodeID,length,functionalClass,directionOfTravel,speedCategory,fromRefSpeedLimit,toRefSpeedLimit,fromRefNumLanes,toRefNumLanes,multiDigitized,urban,timeZone,shapeInfo,curvatureInfo,slopeInfo
0,62007637,162844982,162809070,335.04,5,B,7,30,30,0,0,F,T,0.0,51.4965800/9.3862299/|51.4994700/9.3848799/,NaN,NaN
1,567329767,162844982,162981512,134.56,5,B,7,0,0,0,0,F,T,0.0,51.4965800/9.3862299/|51.4966899/9.3867100/|51...,NaN,NaN
2,62007648,162877732,162844982,97.01,5,B,7,30,30,0,0,F,T,0.0,51.4962899/9.3849100/|51.4965800/9.3862299/,NaN,NaN
3,78670326,162877732,163152693,314.84,5,B,7,30,30,0,0,F,T,0.0,51.4962899/9.3849100/|51.4990000/9.3836099/,NaN,NaN
4,51881672,174713859,174587951,110.17,3,B,6,50,50,2,2,F,T,0.0,53.0643099/8.7903400/45.79|53.0650299/8.791470...,NaN,0.00/-0.090|110.17/0.062


#### Initial Observations

As can be seen in the first 4 rows of `link_data`, there are missing values in the data. More specifically, we can see right away that `curvatureInfo` and `slopeInfo` are missing from some rows. After checking the `README`, we confirm that this is expected.